# download GFS forecast data

## import libraries

In [1]:
from datetime import datetime, timedelta
import xarray as xr
%matplotlib inline
from pathlib import Path

## input lon, lat, time to download

In [2]:
lon_min = 31
lon_max = 44
lat_min = 11
lat_max = 31
datestr_start = "2022-10-12"
datestr_end = "2022-10-17"

outdir = "/Users/backeb/Documents/data/LOST_use_cases/yemen_sunk"

## get GFS forecast data
* fetch data for each day and write to `tmp/gfs_YYYY-MM-DD.nc` 
* except if today, then fetch forecast data from today to end of forecast window

In [3]:
delta = datetime.strptime(datestr_end, "%Y-%m-%d") - datetime.strptime(datestr_start, "%Y-%m-%d")

for i in range(delta.days+1):
    day = datetime.strptime(datestr_start, "%Y-%m-%d") + timedelta(days=i)
    Path(outdir+'/gfs/tmp').mkdir(parents=True, exist_ok=True)
    if day.date() < datetime.now().date():
        url = "http://nomads.ncep.noaa.gov:80/dods/gfs_0p25/gfs"+day.strftime("%Y%m%d")+"/gfs_0p25_00z"
        ds = xr.open_dataset(url)\
                [["ugrd10m","vgrd10m"]]\
                .sel(\
                    lon=slice(lon_min, lon_max), 
                    lat=slice(lat_min, lat_max),
                    time=slice(day.strftime("%Y-%m-%d")+"T00:00:00.000000000",day.strftime("%Y-%m-%d")+"T23:59:59.000000000")\
                    )
    elif day.date() == datetime.now().date():
        url = "http://nomads.ncep.noaa.gov:80/dods/gfs_0p25/gfs"+day.strftime("%Y%m%d")+"/gfs_0p25_00z"
        ds = xr.open_dataset(url)\
            [["ugrd10m","vgrd10m"]]\
                .sel(\
                    lon=slice(lon_min, lon_max), 
                    lat=slice(lat_min, lat_max)
                    )
    else:
        break
    ds.to_netcdf(outdir+"/gfs/tmp/gfs_"+day.strftime("%Y-%m-%d")+".nc")

/opt/miniconda3/lib/python3.9/site-packages/xarray/coding/times.py:123: SerializationWarning: Ambiguous reference date string: 1-1-1 00:00:0.0. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0001-1-1 00:00:0.0). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)
/opt/miniconda3/lib/python3.9/site-packages/xarray/coding/times.py:123: SerializationWarning: Ambiguous reference date string: 1-1-1 00:00:0.0. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0001-1-1 00:00:0.0). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)
/opt/miniconda3/lib/python3.9/site-packages/xarray/coding/times.py:123: SerializationWarning: Ambiguous 

load all `tmp/gfs_YYYY-MM-DD.nc` and write to `gfs.nc`

In [ ]:
ds = xr.open_mfdataset(outdir+"/gfs/tmp/gfs_*.nc")
ds.to_netcdf(outdir+"/gfs/gfs.nc")

check `gfs.nc`

In [8]:
ds = xr.open_dataset(outdir+"/gfs/gfs.nc")
ds

<xarray.Dataset>
Dimensions:  (time: 137, lat: 81, lon: 53)
Coordinates:
  * time     (time) datetime64[ns] 2022-10-12 2022-10-12T03:00:00 ... 2022-10-29
  * lat      (lat) float64 11.0 11.25 11.5 11.75 12.0 ... 30.25 30.5 30.75 31.0
  * lon      (lon) float64 31.0 31.25 31.5 31.75 32.0 ... 43.25 43.5 43.75 44.0
Data variables:
    ugrd10m  (time, lat, lon) float32 ...
    vgrd10m  (time, lat, lon) float32 ...
Attributes:
    title:        GFS 0.25 deg starting from 00Z12oct2022, downloaded Oct 12 ...
    Conventions:  COARDS\nGrADS
    dataType:     Grid
    history:      Wed Oct 12 07:53:17 UTC 2022 : imported by GrADS Data Serve...

In [ ]:
ds.ugrd10m.isel(time=-2).plot.contourf()
#ds.time.plot(marker='.')


In [ ]:
# list directories in a URL

from bs4 import BeautifulSoup
import requests
import re

base_url = "http://nomads.ncep.noaa.gov:80/dods/gfs_0p25/"

r = requests.get(base_url)

soup = BeautifulSoup(r.text, "html.parser")
#parent = [base_url + node.get('href') for node in soup.find_all('a') if node.get('href')]
folders = soup.find_all(text=re.compile("2022"))


In [ ]:
day = datetime.strptime(datestr_start, "%Y-%m-%d") + timedelta(days=5)
print(day.date())
day.date() < datetime.now().date()